# SUPERSTORE PROJECT
by: Kristofer Rivera, Oliver Ton - 04/21/2022

## Overview

Superstore's mission is to be the preferred supplier of workspace solutions; from home-office to cooperate office, we aspire to be the leading expert in workplace solutions for everyone! For this reason, it is important to know whether we are reaching everyone with our products and services. This project will use exploration, modeling to identify the best category for Superstore in regards to sales and will provide recommendations on where to shift our company focus in order to maintain happy loyal customers while continuing to grow our customer base.

## Goal

The goal of this project is to identify which category is the best for SuperStore, a office-supply retail store, and make recommendations on where to shift company focus in regards to category.



## Focused Questions

1.What category base contributes the most to profit?

2.What category base contributes the most to sales?

3.How does the impact of each customer segment change over time?

4.What is the sales goals for the next 5 years?

In [2]:
# visualize 
import matplotlib.pyplot as plt

# for tsa
import statsmodels.api as sm

#important import 
import pandas as pd
import numpy as np
import os
import wrangle as w
import model as m

# plotting defaults
plt.rc('figure', figsize=(13, 7))
plt.style.use('seaborn-dark-palette')
plt.rc('font', size=16)

### Wrangle

In [4]:
#Acquire superstore dataset
df = w.get_superstore_data()

In [6]:
#Clean superstore data
df = w.prep_superstore(df, 'Order Date', 'Ship Date')

**Take away:**
- The data have no nulls, change datatypes of order date and ship date into datetime and then set order date as an index.
- Remove id columns (foreign keys)
- Rename columns, create month columns (month name)

### Exploration

## Modeling

### Conclusion

### Recomendations

### Next steps